In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import io
#불러올 파일의 경로를 filename 변수에 저장
filename = '/content/drive/MyDrive/mbti_1.csv'

In [ ]:
#pandas read_csv로 불러오기
data = pd.read_csv(filename)
data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
data.posts.values[0]

"'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-home-design.jpg ...

In [ ]:
import string
import re

def clean_text(text):
    regex=re.compile('[%s]' % re.escape('|'))                             
    text=regex.sub(" " ,text)                                            
    words=str(text).split()
    words=[i.lower()+ " " for i in words] 
    words=[i for i in words if not "http" in i] 
    words = " ".join(words)
    words = words.translate(words.maketrans('', '', string.punctuation))
    return words

In [ ]:
types = np.unique(data.type.values)           #mbti 타입 16개만 가짐(8600개의 mbti 타입중 16개만 가짐 )

In [ ]:
def get_type_index(string):
    return list(types).index(string)                          #위의 16개의 mbti 유형 값들을 리스트화해서 인덱스를 반납
                                                            #각각의 유형에 0부터 15까지 인덱스가 붙음

In [ ]:
data['type_index'] = data['type'].apply(get_type_index)         #데이터 type을 get type index 함수를 이용해 type index 값을 생성
                                                            #post 부분 보면 type_index 추가된거 확인 가능

In [ ]:
data['cleaned_text']=data['posts'].apply(clean_text)               #데이터 post를 clean text 함수를 이용해 cleaned_text 생성
                                                                       #post 부분 보면 cleaned_text 추가된거 확인가능

In [ ]:
data.cleaned_text.values[0]

'enfp  and  intj  moments  sportscenter  not  top  ten  plays  pranks  what  has  been  the  most  lifechanging  experience  in  your  life  on  repeat  for  most  of  today  may  the  perc  experience  immerse  you  the  last  thing  my  infj  friend  posted  on  his  facebook  before  committing  suicide  the  next  day  rest  in  peace  hello  enfj7  sorry  to  hear  of  your  distress  its  only  natural  for  a  relationship  to  not  be  perfection  all  the  time  in  every  moment  of  existence  try  to  figure  the  hard  times  as  times  of  growth  as  84389  84390    welcome  and  stuff  game  set  match  prozac  wellbrutin  at  least  thirty  minutes  of  moving  your  legs  and  i  dont  mean  moving  them  while  sitting  in  your  same  desk  chair  weed  in  moderation  maybe  try  edibles  as  a  healthier  alternative  basically  come  up  with  three  items  youve  determined  that  each  type  or  whichever  types  you  want  to  do  would  more  than  likely  us

In [ ]:
data.head()

,type,posts,type_index,cleaned_text
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,enfp and intj moments sportscenter not t...
1,ENTP,'I'm finding the lack of me in these posts ver...,3,im finding the lack of me in these pos...
2,INTP,'Good one _____ https://www.youtube.com/wat...,11,good one of course to which i say i ...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,dear intp i enjoyed our conversation the...
4,ENTJ,'You're fired.|||That's another silly misconce...,2,youre fired thats another silly misconcep...


In [ ]:
from sklearn.model_selection import train_test_split

train, test=train_test_split(data)                               #데이터를 train, test 로 나눔
train, val=train_test_split(train)                               # train 데이터를 train, valid로 나눔


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size=10000                                                       #사용할 단어 집합의 크기
trunc_type="post"
pad_type="post"
oov_tok="<OOV>"  # oov_token: 값을 주게 되면, 해당 값은 단어집합에 없는 단어를 대체할 word_index가 된다
 # 함수 보면 단어 집합 크기 만개인데 만개의 원소 안에 없는 단어가 나오면 인덱스 값을 그냥 1로 할당
                                                   
tokenizer=Tokenizer(num_words=vocab_size, oov_token=oov_tok)          #가장 빈도수가 높은 10000개 단어만 선택/oov_token
tokenizer.fit_on_texts(data.cleaned_text.values)                      #단어 인덱스 구축


In [ ]:
maxlen=1500
train_sequences=tokenizer.texts_to_sequences(train.cleaned_text.values)
train_padded=pad_sequences(train_sequences, maxlen=maxlen, truncating=trunc_type,padding=pad_type)  #시퀀스를 패딩시킴
#truncating: 시퀀스 길이가 maxlen 보다 길경우 어느 위치에서 자를 것인지 결정(pre or post)
#padding:어느 위치에 padding 할건지(pre or post)
val_sequences=tokenizer.texts_to_sequences(val.cleaned_text.values)
val_padded=pad_sequences(val_sequences, maxlen=maxlen, truncating=trunc_type,
                        padding=pad_type)

In [ ]:
train_padded

array([[   1,   99,  120, ...,    0,    0,    0],
       [   2,   28,  981, ...,    0,    0,    0],
       [  81,   12,    3, ...,    0,    0,    0],
       ...,
       [1772,    2,  437, ...,    0,    0,    0],
       [   2,   38,    2, ...,    8,   16,   44],
       [   2,  307,   12, ...,    0,    0,    0]], dtype=int32)

In [ ]:
one_hot_labels=tf.keras.utils.to_categorical(train.type_index.values, num_classes=16)  #원핫인코딩
val_labels=tf.keras.utils.to_categorical(val.type_index.values, num_classes=16)


In [ ]:
from tensorflow.keras.models import Sequential                  #모델 설계에 필요한 함수 불러오기
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten, Dropout, Conv1D, GlobalMaxPooling1D,TimeDistributed
# 임베딩층, dropout(overfitting 해결하기 위해 층의 일부 제거), bidirectional lstm(양방향 lstm),dense
def create_model():
    op=tf.keras.optimizers.Adam()
    model=Sequential()                #sequential 함수를 모델로 설정
    model.add(Embedding(vocab_size, 256, input_length=maxlen)) 
    #임베딩층 추가, 워드 임베딩 후의 임베딩 차원256(노드 256개),입렵값의 길이크기
    model.add(Dropout(0.3))             #dropout layer-오버피팅 해결책(확률은 0.3)                                 
    model.add(Bidirectional(LSTM(50,return_sequences=True)))      #텍스트데이터는 경우에 따라 역방향 추론이 효율적일 수 있음   
                    #dromout layer-오버피팅 해결책(확률은 0.3)                 
    model.add (TimeDistributed(Dense(50)))
    
    model.add(Bidirectional(LSTM(50)))       #텍스트데이터는 경우에 따라 역방향 추론이 효율적일 수 있음  
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))#노드 64개  활성함수 relu #활성함수 relu 은닉층으로 학습함, 역전파를 통해 좋은 성능 기대
    model.add(Dense(16, activation= 'softmax'))#노드 16개 -> mbti 유형 16개  활성함수 softmax #활성함수 softmax 클래스 분류할떄

    model.compile(loss='categorical_crossentropy', optimizer=op, metrics=['accuracy']) 
    #모델 컴파일 손실함수 지정(손실값을 최소화하는 방향으로 가중치 수정)
    # optimizer 학습 최적화를 위해서 사용
    # 모델 평가 기준 정확도
    return model

In [ ]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()
    
model.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.12.32.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.12.32.242:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1500, 256)         2560000   
_________________________________________________________________
dropout (Dropout)            (None, 1500, 256)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 1500, 100)         122800    
_________________________________________________________________
time_distributed (TimeDistri (None, 1500, 50)          5050      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               40400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                6

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1500, 256)         2560000   
_________________________________________________________________
dropout (Dropout)            (None, 1500, 256)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 1500, 100)         122800    
_________________________________________________________________
time_distributed (TimeDistri (None, 1500, 50)          5050      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               40400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                6

In [ ]:
model.fit(train_padded, one_hot_labels, epochs =20, verbose = 1, 
          validation_data = (val_padded, val_labels))


Epoch 1/20
  2/153 [..............................] - ETA: 18s - loss: 0.9841 - accuracy: 0.7344WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0020s vs `on_train_batch_end` time: 0.1178s). Check your callbacks.


153/153 [==============================] - ETA: 0s - loss: 0.9473 - accuracy: 0.6962WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0015s vs `on_test_batch_end` time: 0.0304s). Check your callbacks.


153/153 [==============================] - 21s 135ms/step - loss: 0.9473 - accuracy: 0.6962 - val_loss: 3.5760 - val_accuracy: 0.1739
Epoch 2/20
153/153 [==============================] - 20s 134ms/step - loss: 0.7197 - accuracy: 0.7692 - val_loss: 4.0182 - val_accuracy: 0.1604
Epoch 3/20
153/153 [==============================] - 21s 134ms/step - loss: 0.5561 - accuracy: 0.8313 - val_loss: 4.3965 - val_accuracy: 0.1543
Epoch 4/20
153/153 [==============================] - 21s 136ms/step - loss: 0.4352 - accuracy: 0.8600 - val_loss: 4.7452 - val_accuracy: 0.1838
Epoch 5/20
153/153 [==============================] - 21s 135ms/step - loss: 0.3367 - accuracy: 0.8944 - val_loss: 5.1636 - val_accuracy: 0.1758
Epoch 6/20
153/153 [==============================] - 20s 134ms/step - loss: 0.2663 - accuracy: 0.9135 - val_loss: 5.4661 - val_accuracy: 0.1709
Epoch 7/20
153/153 [==============================] - 21s 134ms/step - loss: 0.2344 - accuracy: 0.9266 - val_loss: 5.7028 - val_accuracy: 0.1